## Data Preparation

In [1]:
from datasets import load_dataset
import numpy as np

import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Dataset

In [2]:
dataset = load_dataset("imagefolder", data_dir="./datasets/chest_xray")

Resolving data files: 100%|██████████| 624/624 [00:00<00:00, 312320.49it/s]


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


### Setup Labels

In [4]:
labels = labels = dataset["train"].features["label"].names
print(labels)

['NORMAL', 'PNEUMONIA']


In [5]:
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
  label2id[i] = label
  id2label[label] = i

In [6]:
print(label2id)
print(id2label)

{0: 'NORMAL', 1: 'PNEUMONIA'}
{'NORMAL': 0, 'PNEUMONIA': 1}


### Transforming Data

In [7]:
from transformers import AutoImageProcessor
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

In [8]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [9]:
size = (image_processor.size["height"], image_processor.size["width"])
resizer = RandomResizedCrop(size)
normalize = Normalize(image_processor.image_mean, image_processor.image_std)

In [10]:
_transforms = Compose([resizer, ToTensor(), normalize])

In [11]:
def transforms(examples):
  examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
  del examples["image"]
  return examples

In [12]:
dataset  = dataset.with_transform(transforms)

In [13]:
print(dataset['train'])

Dataset({
    features: ['image', 'label'],
    num_rows: 5216
})


### Preparing metrics for the model

In [14]:
accuracy = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
  predictions = np.argmax(eval_pred.predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

### Setting Up Model

In [16]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
  "google/vit-base-patch16-224-in21k",
  num_labels=len(labels),
  id2label=id2label,
  label2id=label2id
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model = model.to("cuda")

In [18]:
model.device

device(type='cuda', index=0)

### Training The Model

In [19]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DefaultDataCollator

In [20]:
training_args = TrainingArguments(
  output_dir = "pneumonia_model",
  evaluation_strategy="epoch",
  save_strategy="epoch",
  learning_rate=5e-5,
  per_device_train_batch_size=12,
  per_device_eval_batch_size=12,
  num_train_epochs=10,
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  remove_unused_columns=False,
)

In [21]:
trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=DefaultDataCollator(),
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"],
  tokenizer=image_processor,
  compute_metrics=compute_metrics
)

In [22]:
trainer.train()

                                                  
 10%|█         | 435/4350 [03:13<23:10,  2.82it/s]

{'eval_loss': 0.37469497323036194, 'eval_accuracy': 0.8798076923076923, 'eval_runtime': 14.3549, 'eval_samples_per_second': 43.469, 'eval_steps_per_second': 3.622, 'epoch': 1.0}


 11%|█▏        | 500/4350 [03:40<26:16,  2.44it/s]  

{'loss': 0.248, 'learning_rate': 4.4252873563218394e-05, 'epoch': 1.15}


                                                  
 20%|██        | 870/4350 [06:20<19:38,  2.95it/s]

...

                                                      
100%|██████████| 4350/4350 [45:06<00:00,  2.70it/s]

{'eval_loss': 0.2847822308540344, 'eval_accuracy': 0.9118589743589743, 'eval_runtime': 13.1616, 'eval_samples_per_second': 47.411, 'eval_steps_per_second': 3.951, 'epoch': 10.0}


100%|██████████| 4350/4350 [45:07<00:00,  1.61it/s]

{'train_runtime': 2707.5956, 'train_samples_per_second': 19.264, 'train_steps_per_second': 1.607, 'train_loss': 0.1291817969837408, 'epoch': 10.0}


TrainOutput(global_step=4350, training_loss=0.1291817969837408, metrics={'train_runtime': 2707.5956, 'train_samples_per_second': 19.264, 'train_steps_per_second': 1.607, 'train_loss': 0.1291817969837408, 'epoch': 10.0})

In [23]:
dataset_val = dataset['validation'][:]

dataset_val['label']

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [24]:
# image = dataset_val["image"]

# dataset_val['pixel_values']

In [25]:
model.to('cpu')

for i in range(0, 16):  
  image = dataset_val["pixel_values"][i]
  
  pred = model(image[None, ...])
  
  logits = pred.logits.detach().numpy()[0]
  pred_class = np.argmax(logits)
  
  print(logits, pred_class)

[ 1.0386438 -0.986711 ] 0
[-0.13800992  0.19367892] 1
[ 0.20198247 -0.15564975] 0
[-0.95488036  0.848078  ] 1
[-1.2200586  1.0992448] 1
[-0.5879604  0.5493035] 1
[ 1.4296321 -1.0805426] 0
[ 2.228238  -2.0731947] 0
[-2.9751143  2.8376467] 1
[-2.969958  2.867535] 1
[-1.8585757  1.7708261] 1
[-2.1129858  2.0069702] 1
[-2.2598677  2.1570873] 1
[-2.9136753  2.78434  ] 1
[-2.8000977  2.695844 ] 1
[-2.7378416  2.681501 ] 1


###